In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("./data/3_train.csv")
data.head(5)

,ID,area,ck,ck-saveall,ck-drawall,ck-savetime,ck-drawtime,ck-saveavg,ck-drawavg,ck-avg,...,dep-avg,dep-9201,fed-9201,fed-avg,comp,ck-changame,dep-changame,ck-changtime,dep-changtime,VV
0,3,709,0,?,NaN,?,?,0,NaN,?,...,0,0,NaN,0,0,0,0,0,0,0
1,4,.,NaN,0,0,0,NaN,0,0,?,...,0,?,?,0,0,0,0,0,0,0
2,5,616,?,0,.,0,0,0,NaN,0,...,0,NaN,0,.,0,0,0,0,.,0
3,6,615,0,0,0,?,NaN,?,0,0,...,0,0,0,NaN,0,0,0,0,0,0
4,7,.,0,0,0,NaN,.,?,0,NaN,...,0,0,0,0,?,0,0,?,0,0


In [3]:
from Python.pandas.feature_selection import *
object_feature_helper(data)


---------------------------------------------
data contains [ 26144 ] records 
ID do not have any null values.
ID  is numeric column:
count    26144.000000
mean     16333.478045
std       9429.671054
min          3.000000
25%       8139.250000
50%      16340.500000
75%      24498.250000
max      32682.000000
Name: ID, dtype: float64



---------------------------------------------
area have 1721 null values:

area  has [ 489 ] different values, please create features yourself!
please notice here are some seldom_indexes:  Index(['815', '23700', '840', '90900', '260', '90800', '545', '826', '638',
       '974',
       ...
       '605', '98300', '544', '26500', '31200', '263', '513', '74500', '223',
       '270'],
      dtype='object', length=253)



---------------------------------------------
ck have 1641 null values:

0    12749
1     6717
?     3370
.     1667
Name: ck, dtype: int64
use following code to generate features from label mapping:
ck_mapping = {'0': 0, '1': 1, '?': 2, '.':

In [4]:
def area_handler(area):
    if area in ['.', 'nan', '?']:
        return 0
    if len(area) < 3:
        return str(area) + "00"
    return str(area)[:3]

data_area = data['area'].apply(lambda area: area_handler(str(area)))
data_area1 = data_area.apply(lambda area: str(area)[:1])
from sklearn.preprocessing import OneHotEncoder
area_ohe = OneHotEncoder(drop='first')
data_area1_ohe = area_ohe.fit_transform(data_area1.values.reshape(-1,1))
data_area1_ohe = pd.DataFrame(data_area1_ohe.toarray(), index=data.index)
data_area1_ohe


,0,1,2,3,4,5,6,7,8
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
26139,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
26141,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ck_mapping = {'0': 0, '1': 1, '?': 0, '.': 0}
data_ck, ck_ohe = one_hot_encoder_column(data, 'ck', ck_mapping, fill_na=0)

In [6]:
def numeric_to_int(x, na_val=-1):
    if x is None:
        return na_val
    if type(x) == str:
        if x in ['?', '.', 'NaN']:
            return na_val
        return int(x)
    if np.isnan(x):
        return na_val
    return x
def multi_fit(x, y, z):
    newx = numeric_to_int(x)
    newy = numeric_to_int(y)
    newz = numeric_to_int(z)
    if newx == -1:
        if newy != -1 and newz != -1:
            newx = newy * newz

    if newy == -1:
        if newx != -1 and newz > 0:
            newy = newx / newz
    if newz == -1:
        if newx != -1 and newy > 0:
            newz = newx / newy

    return newx, newy, newz
data_ck_saveall, data_ck_savetime, data_ck_saveavg = \
    zip(*data.apply(lambda row: multi_fit(row['ck-saveall'], row['ck-savetime'], row['ck-saveavg']), axis=1))
data_ckall = pd.concat([pd.Series(data_ck_saveall), pd.Series(data_ck_savetime), pd.Series(data_ck_saveavg)], axis=1)

In [7]:
data_dk_saveall, data_dk_savetime, data_dk_saveavg = \
    zip(*data.apply(lambda row: multi_fit(row['ck-drawall'], row['ck-drawtime'], row['ck-drawavg']), axis=1))
data_dkall = pd.concat([pd.Series(data_dk_saveall), pd.Series(data_dk_savetime), pd.Series(data_dk_saveavg)], axis=1)

In [8]:
data_ck_avg = data['ck-avg'].apply(lambda x: numeric_to_int(x, na_val=0))
data_ck_avg

0           0
1           0
2           0
3           0
4           0
         ... 
26139       0
26140       0
26141       0
26142       0
26143    1149
Name: ck-avg, Length: 26144, dtype: int64

In [9]:
data_dep_avg = data['dep-avg'].apply(lambda x: numeric_to_int(x, na_val=0))
data_dep_avg

0            0
1            0
2            0
3            0
4            0
         ...  
26139     1443
26140    25566
26141     2173
26142        0
26143        0
Name: dep-avg, Length: 26144, dtype: int64

In [10]:
data_dep9201 = data['dep-9201'].apply(lambda x: numeric_to_int(x, na_val=0))
data_fed9201 = data['fed-9201'].apply(lambda x: numeric_to_int(x, na_val=0))
data_fed_avg = data['fed-avg'].apply(lambda x: numeric_to_int(x, na_val=0))

In [11]:
data_ck_changame = data['ck-changame'].apply(lambda x: numeric_to_int(x, na_val=0))
data_dep_changame = data['dep-changame'].apply(lambda x: numeric_to_int(x, na_val=0))
data_ck_changtime = data['ck-changtime'].apply(lambda x: numeric_to_int(x, na_val=0))
data_dep_changtime = data['dep-changtime'].apply(lambda x: numeric_to_int(x, na_val=0))

In [12]:
comp_mapping = {'0': 0, '?': 0, '1': 1, '.': 0}
data_comp, comp_ohe = one_hot_encoder_column(data, 'comp', comp_mapping, fill_na=0)

In [13]:
data_y = data['VV']

data_all = pd.concat([data_area1_ohe, data_ck, data_ckall, data_dkall, data_ck_avg, data_dep_avg,
                      data_dep9201, data_fed9201, data_fed_avg, data_ck_changame, data_dep_changame,
                      data_ck_changtime, data_dep_changtime, data_comp], axis=1)
data_all

,0,1,2,3,4,5,6,7,8,ck_0,...,ck-avg,dep-avg,dep-9201,fed-9201,fed-avg,ck-changame,dep-changame,ck-changtime,dep-changtime,comp_0
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26139,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1443,0,0,0,0,0,0,0,0.0
26140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,25566,0,0,0,0,0,0,0,0.0
26141,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,2173,0,0,0,0,0,0,0,0.0
26142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0


In [14]:
data_all.to_csv("./data/feature3_train.csv")